In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, sys
sys.path.append('/content/drive/MyDrive/CPD_BT')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import random

import itertools

from bt_cpd import *

import time
import bisect

import pandas as pd

import statsmodels.api as sm
from sklearn import linear_model

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
T = 4
Delta = 2000
m = np.array([Delta] * T)
cp_truth = np.cumsum(m)[:T-1]

n = 100
beta = np.zeros((T, n))

t = 0.9
kappa = np.log(t / (1 - t))
delta = 1
beta_ref = get_beta_with_gap(n, delta)
beta_ref *= kappa / (np.max(beta_ref) - np.min(beta_ref))
beta[0] = beta_ref[:]
beta[1] = change_type(beta_ref, 1)
beta[2] = change_type(beta_ref, 2)
beta[3] = change_type(beta_ref, 3)

print(max(beta[0]) - min(beta[0]))

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(beta[t] - beta[t - 1])**2)**0.5
print(diff)

2.1972245773362196
[12.81317951 11.09709382 12.81317951]


In [5]:
cp_truth

array([2000, 4000, 6000])

In [6]:
path = '/content/drive/MyDrive/CPD_BT/'
with open(path + 'data_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '.npy', 'rb') as f:
    X_train_list = np.load(f)
    Y_train_list = np.load(f)
    X_test_list = np.load(f)
    Y_test_list = np.load(f)

In [7]:
X_train_list.shape

(100, 8000, 100)

In [8]:
np.random.seed(0)

grid_n = 100
gamma_list = [80, 100]
lam_list = [0.1]

nt = Delta * T
B = 100

run_time_d = np.zeros(B)
loc_error_d = np.zeros(B)
K_d = np.zeros(B)

cp_best_list = []
param_best_list = []
cp_best_cand_list = []

for b in range(B):
    X_train = X_train_list[b]
    Y_train = Y_train_list[b]
    X_test = X_test_list[b]
    Y_test = Y_test_list[b]

    start_time = time.time()
    dp_fit = dplr_cv_bt(grid_n, lam_list, gamma_list)
    cp_best, param_best, cp_best_cand = dp_fit.fit((Y_train, X_train), (Y_test, X_test))    
    run_time_d[b] = time.time() - start_time
    loc_error_d[b] = cp_distance(cp_best, cp_truth)
    K_d[b] = len(cp_best)

    cp_best_list.append(cp_best)
    param_best_list.append(param_best)
    cp_best_cand_list.append(cp_best_cand)
    print(b)


print('---------- dplr -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_d.mean(), run_time_d.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_d.std(), run_time_d.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_d < T - 1), sum(K_d == T - 1), sum(K_d > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- dplr -----------
avg loc error: 12.44, avg time: 402.4263724064827
std loc error: 12.065918945525866, std time: 7.381561277883988
K < K*: 0, K = K*: 100, K > K*: 0


In [9]:
loc_error_d

array([25.,  2., 12., 19.,  8.,  4.,  9., 14.,  2., 10., 56., 14., 10.,
        1.,  5.,  6.,  5., 38.,  8., 15.,  9., 13., 11., 14., 13., 24.,
        2.,  7.,  3.,  5.,  5., 10.,  5.,  8., 16., 25.,  4., 11., 11.,
       66.,  7.,  6.,  7., 10.,  3.,  1., 11.,  4., 10., 22.,  5., 13.,
        4., 20., 22.,  7., 12.,  4.,  7.,  3., 19.,  2., 12., 50., 12.,
        4.,  6., 13.,  9., 10., 32.,  6.,  3.,  9.,  5.,  3., 16., 10.,
       24.,  2., 12., 14.,  4., 61.,  4., 42., 15.,  8., 10., 22.,  3.,
       15., 15.,  6., 15.,  3., 11., 11., 19.,  4.])

In [10]:
cp_best_list

[[1975, 3999, 5996],
 [2000, 4002, 6002],
 [1988, 4001, 5993],
 [2005, 3985, 6019],
 [1992, 4008, 6000],
 [1996, 4000, 5999],
 [2000, 4009, 6006],
 [2005, 4007, 5986],
 [2001, 4000, 6002],
 [1997, 4010, 5996],
 [1993, 4056, 5983],
 [1993, 4009, 6014],
 [2001, 3994, 5990],
 [1999, 3999, 5999],
 [2003, 4003, 6005],
 [2000, 4001, 5994],
 [2005, 4000, 5997],
 [1998, 3962, 5995],
 [2008, 3999, 5996],
 [2015, 4000, 5998],
 [2006, 3991, 5998],
 [2013, 3994, 6000],
 [1998, 3991, 5989],
 [1994, 3996, 5986],
 [2013, 3999, 6000],
 [1995, 3976, 6000],
 [1999, 4002, 5998],
 [1995, 4002, 6007],
 [1997, 4003, 5999],
 [2000, 3997, 6005],
 [2002, 3996, 5995],
 [2006, 4000, 6010],
 [2005, 4005, 6001],
 [1995, 4001, 5992],
 [1996, 3984, 6001],
 [1975, 3980, 6000],
 [2000, 3997, 6004],
 [2011, 4006, 5999],
 [1999, 3989, 6007],
 [2001, 4066, 6004],
 [2005, 3993, 5999],
 [2006, 4002, 6003],
 [2001, 3995, 5993],
 [2000, 3990, 6002],
 [2003, 3999, 5999],
 [2001, 4000, 6000],
 [2011, 4000, 5997],
 [2001, 3996,

In [11]:
import pickle
with open(path + 'dplr_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_grid' + str(grid_n) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, param_best_list, cp_best_cand_list, loc_error_d, run_time_d, K_d], f)